## Функция hash() и хэши объектов

В Python есть специальная функция hash, позволяющая вычислять хэш по определенному алгоритму для неизменяемых объектов. Для равных объектов на выходе будет получаться одинаковый хэш.

К примеру, можно вычислить хэш целого числа:

In [1]:
hash(123)

123

Или строки:

In [2]:
hash('Python')

-3742965780803970851

И если снова вычислить хэш для этой строки, результат будет идентичным:

In [3]:
hash('Python')

-3742965780803970851

Аналогично и с кортежем:

In [4]:
hash((1, 2, 3))

529344067295497451

In [5]:
hash((1, 2, 3))

529344067295497451

Т.е. для одинаковых объектов хэши одинаковы. Но обратное утвержение неверно: если хэши одинаковые, то это не значит, что объекты одинаковы.

Если объекты a == b (равны), то равен и их хэш. <br>
Равные хэши hash(a) == hash(b) не гарантируют равенство объектов.<br>
Если хэши не равны: hash(a) != hash(b), то объекты точно не равны.<br>

Хэши можно вычислять только для неизменяемых объектов. Если попытаться выхвать хэш, к примеру, для списка, то будет ошибка: unhashable type. Изменяемые объекты являются нехэшируемыми.

Некоторые объекты в Python (например, словари) используют хэш в качестве своих ключей. Когда у словаря прописывается какой либо ключ - он должен быть не изменяемым объектом. Если попробовать использовать в качестве ключа изменяемый объект - будет ошибка. Словарь хранит ключи в виде коллекции: (хэш ключа, ключ). Это нужно для оптимизации поиска - реализован алгоритм, который быстро это делает именно при помощи хэша. Если вдруг встречаются равные хэши - то алгоритм смотрит на ключ, после чего поиск нужного значения завершается.

Объекты пользовательского класса считаются неизменяемыми, и для них можно вычислить хэш:

In [6]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

Не смотря на то, что два объекта с одинаковыми параметрами:

In [7]:
p1 = Point(1, 2)
p2 = Point(1, 2)

Хэши у них разные:

In [8]:
print(hash(p1), hash(p2), sep='\n')

111260066118
111260066292


Функция понимает, что объекты разные благодаря такому сравнению:

In [9]:
p1 == p2

False

Так как если объекты равны - их хеш должен быть одинаковым.

Но если определить магический метод eq, который отвечает за сравнение объектов:

In [10]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __eq__(self, other):
        return (self.x == other.x and self.y == other.y)

In [11]:
p1 = Point(1, 2)
p2 = Point(1, 2)

In [12]:
p1 == p2

True

То функция hash работать перестанет. Для того, чтобы восстановить ее работу, нужно переопределить магический метод hash, который и вызывается при вычислении хэша, а внутри вызвать фунцию hash, но уже не от самого объекта, а от его параметров:

In [13]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __eq__(self, other):
        return (self.x == other.x and self.y == other.y)
    
    def __hash__(self):
        return hash((self.x, self.y))

In [14]:
p1 = Point(1, 2)
p2 = Point(1, 2)

In [15]:
print(hash(p1), hash(p2), sep='\n')

-3550055125485641917
-3550055125485641917


И хэши этих объектов будут равны, так как параметры этих объектов равны.

Теперь если сформировать словарь:

In [16]:
d = {}

И в качестве ключей указать эти объекты:

In [17]:
d[p1] = 1
d[p2] = 2

То в этом словаре будет один ключ со значением 2:

In [18]:
d

{<__main__.Point at 0x19e79e3b8b0>: 2}

Это говорит о том, что объекты p1 и p2 воспринимаются как один и тот же ключ, так как их хэши равны и оператор сравнения на равенство этих объектов также выдает True: то есть для словаря это один и тот же ключ.